In [2]:
!pip install gradio transformers ollama

  Obtaining dependency information for httpx>=0.24.1 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-any.whl.metadata
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.3.2 requires httpx~=0.28.0, but you have httpx 0.27.2 which is incompatible.


In [3]:
import gradio as gr
from transformers import pipeline
import ollama


In [4]:
# Initialize Hugging Face question-answering model
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [9]:
def answer_question(question):
    # Predefined documents or knowledge base (could be data from the RAG system)
    knowledge_base = {
        "How can I navigate to a specific pose, including replanning aspects?": """
        To navigate to a specific pose in ROS2, you first need to define the target pose 
        using the navigation stack. This can be done through the MoveBase action client 
        or by publishing a target pose to the `/move_base_simple/goal` topic. Once the 
        robot reaches the target pose, replanning is necessary when it encounters an obstacle 
        or a deviation from the expected trajectory. This can be achieved by adjusting the 
        costmap or invoking the replanning services in Nav2."""
    }

    # Check if the question is in the knowledge base
    if question in knowledge_base:
        return knowledge_base[question]

    # If not found in the knowledge base, use the model (Hugging Face or Ollama)
    if 'replanning' in question.lower():
        # Example with Hugging Face model
        context = knowledge_base["How can I navigate to a specific pose, including replanning aspects?"]
        result = qa_pipeline(question=question, context=context)
        return result['answer']



In [10]:
# List of predefined questions for the dropdown
questions = [
    "How can I navigate to a specific pose, including replanning aspects?",
    "Can you provide an example of setting up ROS2 navigation?",
    "How do I move a robot using MoveIt2?",
    "What is the best way to simulate robots in Gazebo?"
]

# Create Gradio Interface
def gradio_interface(question):
    return answer_question(question)

interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Dropdown(choices=questions, label="Select a Question"),
    outputs="text",  # Output will be text
    live=True,
    title="RAG System for ROS2 Navigation",
    description="Select a question from the dropdown to get answers about ROS2 navigation, replanning, MoveIt2, and more."
)

# Launch the interface
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
